In [1]:
# Importing modules
import numpy as np
import pandas as pd
import numba
import pyarrow.dataset as ds

In [2]:
# Pandas display options (personal preference)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

In [3]:
# Importing the G09 parquet file
dataset = ds.dataset("../data/WD01_2p4.parquet", format="parquet")
table = dataset.to_table()

# Setting uberID as index
df = table.to_pandas().set_index('uberID')
df.index=df.index.astype('int64')

del dataset,table

In [4]:
# Getting rid of weird spaces in the class column
df=df.replace({'class': {'ambiguous ': 'ambiguous',
                         'star      ': 'star', 
                         'galaxy    ': 'galaxy', 
                         'artefact  ':'artefact'}})

In [5]:
# Making a number of cuts: magnitude, mask, starmask and removing artefacts
df=df[(df['mag_Zt']<22) & 
      (df['mask']==0) & 
      (df['starmask']==0) & 
      (df['class']!='artefact')]

In [6]:
# Getting a 1/10 subset of the data (for the purpose of this tutorial)
# df=df[::10]

In [7]:
df

,FrameName,FrameID,segID,xmax,ymax,censep,RAcen,Deccen,RAmax,Decmax,RAGAIA_r,RAGAIA,DecGAIA,DecGAIA_r,RAGAIA_r_cen,DecGAIA_r_cen,RAcen_gt,Deccen_gt,RAmax_gt,Decmax_gt,RAcen_rt,Deccen_rt,RAmax_rt,Decmax_rt,RAcen_Zt,Deccen_Zt,RAmax_Zt,Decmax_Zt,sky_mean,skyRMS_mean,log10seeing,log10seeing_r,log10seeing_i,log10seeing_Z,log10seeing_Y,mag,EBV,R50,R50_gt,R50_rt,R50_Yt,R50_Jt,R50_Ht,R90,R100,N100,axrat,ang,groupID,Ngroup,mag_app_Zt,mag_Zt,mag_app_gt,mag_app_rt,mag_app_it,mag_app_Yt,flux_ut,flux_err_ut,flux_ut_uncorrected,flux_err_ut_uncorrected,flux_gt,flux_err_gt,flux_gt_uncorrected,flux_err_gt_uncorrected,flux_rt,flux_err_rt,flux_rt_uncorrected,flux_err_rt_uncorrected,flux_it,flux_err_it,flux_it_uncorrected,flux_err_it_uncorrected,flux_Zt,flux_err_Zt,flux_Zt_uncorrected,flux_err_Zt_uncorrected,flux_Yt,flux_err_Yt,flux_Yt_uncorrected,flux_err_Yt_uncorrected,flux_Jt,flux_err_Jt,flux_Jt_uncorrected,flux_err_Jt_uncorrected,flux_Ht,flux_err_Ht,flux_Ht_uncorrected,flux_err_Ht_uncorrected,flux_Kt,flux_err_Kt,flux_Kt_uncorrected,flux_err_Kt_uncorrected,flux_W1t,flux_err_W1t,flux_W1t_uncorrected,flux_err_W1t_uncorrected,flux_W2t,flux_err_W2t,flux_W2t_uncorrected,flux_err_W2t_uncorrected,flux_uc,flux_err_uc,flux_gc,flux_err_gc,flux_rc,flux_err_rc,flux_ic,flux_err_ic,flux_Zc,flux_err_Zc,flux_Yc,flux_err_Yc,flux_Jc,flux_err_Jc,flux_Hc,flux_err_Hc,flux_Kc,flux_err_Kc,flux_W1c,flux_err_W1c,flux_W2c,flux_err_W2c,mask,starmask,starscol,starssize,class,noOPT_r,noOPT_i,noIR_Z,noIR_Y,duplicate,mag_Zc,mag_Zt_fake
uberID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9580882500059,waves_postprocessed_10.0_-42.5.rds,958.0,96,8824.5,58.5,6675.881665,9.791838,-43.078226,9.791855,-43.078250,9.791777,9.791891,-43.078258,-43.078257,9.791801,-43.078256,9.791810,-43.078236,9.791741,-43.078250,9.791765,-43.078248,9.791741,-43.078250,9.791838,-43.078225,9.791855,-43.078250,-0.002034,0.002893,-0.123168,-0.119852,NaN,-0.120193,-0.166938,15.111749,0.013676,0.725778,0.852429,0.776101,0.675511,0.647937,0.653555,2.218231,11.149482,4120,0.949469,41.601561,96,11,15.797645,15.088464,18.469780,17.163015,16.416433,15.483403,2.882643e-05,3.618253e-07,2.713136e-05,3.405490e-07,3.513708e-04,2.522236e-06,3.355205e-04,2.408458e-06,1.155568e-03,4.160529e-07,1.117569e-03,4.023716e-07,0.002289,2.316756e-06,0.002230,2.256998e-06,0.003347,1.348588e-06,0.003282,1.322478e-06,0.004260,3.651365e-06,0.004195,3.596005e-06,0.004631,3.878294e-06,0.004580,3.835726e-06,0.005287,7.656555e-06,0.005249,7.602184e-06,0.004283,5.494495e-06,0.004263,5.469025e-06,0.001973,1.636403e-06,0.001968,1.632260e-06,0.001160,1.389919e-06,0.001158,1.387474e-06,2.679044e-05,2.647858e-07,3.409320e-04,1.008414e-06,1.148247e-03,3.282160e-07,2.284306e-03,1.430991e-06,3.333865e-03,5.246856e-07,4.241085e-03,1.441600e-06,4.614449e-03,1.555188e-06,5.255760e-03,3.291831e-06,4.260607e-03,2.424957e-06,1.856062e-03,4.583290e-07,1.064517e-03,4.881762e-07,0.0,0.0,3.0,3.0,star,0.0,0.0,0.0,0.0,1.0,15.092630,0
9581275900715,waves_postprocessed_10.0_-42.5.rds,958.0,102,12758.5,714.5,8550.818440,9.343646,-43.021880,9.343656,-43.021902,9.343692,9.343692,-43.021909,-43.021909,9.343651,-43.021933,9.343593,-43.021931,9.343656,-43.021902,9.343615,-43.021925,9.343656,-43.021902,9.343647,-43.021879,9.343656,-43.021902,-0.000268,0.002855,-0.123168,-0.119852,NaN,-0.120193,-0.166938,15.445208,0.010967,0.708051,0.788300,0.832923,0.632877,0.626473,0.617132,2.190448,10.772246,3931,0.970473,37.632057,102,7,16.090195,15.425893,18.923794,17.929803,16.862768,15.742158,1.855132e-05,4.099015e-07,1.767128e-05,3.904567e-07,2.082387e-04,4.829733e-07,2.006712e-04,4.654220e-07,6.009600e-04,6.210577e-07,5.850600e-04,6.046259e-07,0.001547,1.745950e-06,0.001515,1.709742e-06,0.002453,1.801823e-06,0.002415,1.773792e-06,0.003164,3.275237e-06,0.003126,3.235355e-06,0.003390,2.071104e-06,0.003360,2.052854e-06,0.003829,1.692155e-06,0.003807,1.682512e-06,0.003332,5.016196e-06,0.0

In [8]:
# Reading the u and no u labels from notebooks 1 and 2, and creating new dataframes
df_u=pd.read_csv('../data/WD01_u.csv').set_index('uberID')
df_no_u=pd.read_csv('../data/WD01_no_u.csv').set_index('uberID')

In [9]:
# Creating new columns in the main dataframe with u and no u labels
df['cluster_label']=np.zeros(len(df))
df['cluster_label_no_u']=np.zeros(len(df))

In [10]:
# Creating umap columns in the main dataframe with u and no u labels
for i in range(10):
    df[f'umap{i+1}_no_u'] = np.zeros(len(df))
    df[f'umap{i+1}_u'] = np.zeros(len(df))

In [11]:
# Filling the new columns with u and no u labels
df.loc[df_u.index,'cluster_label']=df_u['cluster_label'].apply(str)
df.loc[df_no_u.index,'cluster_label_no_u']=df_no_u['cluster_label'].apply(str)

/tmp/ipykernel_218831/2692133000.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['star' 'star' 'star' ... 'galaxy' 'galaxy' 'galaxy']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df_u.index,'cluster_label']=df_u['cluster_label'].apply(str)
/tmp/ipykernel_218831/2692133000.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['star' 'star' 'star' ... 'galaxy' 'galaxy' 'galaxy']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df_no_u.index,'cluster_label_no_u']=df_no_u['cluster_label'].apply(str)


In [12]:
for i in range(10):
    df.loc[df_u.index,f'umap{i+1}_u']=df_u[f'umap{i+1}']
    df.loc[df_no_u.index,f'umap{i+1}_no_u']=df_no_u[f'umap{i+1}']

In [13]:
labels=['star','galaxy']

In [14]:
# Creating a 'final label' column full of 1's to combine the two labels
df['final_label']=np.ones(len(df))


In [15]:
# Creating umap columns with 1's
for i in range(10):
    df[f'umap{i+1}'] = np.ones(len(df))


In [16]:
# First applying the no u labels to the final label
for label in labels:
    df.loc[df[df['cluster_label_no_u']==label].index,'final_label']=label
    # First putting umap columns with no u labels
    for i in range(10):
        df.loc[df[df['cluster_label_no_u']==label].index, f'umap{i+1}'] = df.loc[df[df['cluster_label_no_u']==label].index, f'umap{i+1}_no_u']

/tmp/ipykernel_218831/1789200346.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'star' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df[df['cluster_label_no_u']==label].index,'final_label']=label


In [17]:
# Then applying the u labels to the final label, overwriting any existing no u labels
for label in labels:
    df.loc[df[df['cluster_label']==label].index,'final_label']=label

    for i in range(10):
        df.loc[df[df['cluster_label']==label].index, f'umap{i+1}'] = df.loc[df[df['cluster_label']==label].index, f'umap{i+1}_u']

In [18]:
# Convering final label to string
df['final_label']=df['final_label'].apply(str)

In [19]:
# Creating a new column to indicate whether a source has any missing bands.
# If final label still == 1, that indicates a label hasn't been added, so it has a missing band
df['missing_bands']=True
df.loc[df[df['final_label']!='1.0'].index,'missing_bands']=False

In [20]:
# For the sources with missing labels we then overwrite with class
df.loc[df[df['final_label']=='1.0'].index,'final_label'] = df[df['final_label']=='1.0']['class']

In [31]:
# For sources with missing labels we put nan in the umap columns
for i in range(10):
    df.loc[df[df['missing_bands']==True].index, f'umap{i+1}'] = np.nan

In [32]:
# Sources with missing bands are only 1.5% of the sample
len(df[df['missing_bands']==True])/len(df)

0.05072637011827246

In [33]:
df

,FrameName,FrameID,segID,xmax,ymax,censep,RAcen,Deccen,RAmax,Decmax,RAGAIA_r,RAGAIA,DecGAIA,DecGAIA_r,RAGAIA_r_cen,DecGAIA_r_cen,RAcen_gt,Deccen_gt,RAmax_gt,Decmax_gt,RAcen_rt,Deccen_rt,RAmax_rt,Decmax_rt,RAcen_Zt,Deccen_Zt,RAmax_Zt,Decmax_Zt,sky_mean,skyRMS_mean,log10seeing,log10seeing_r,log10seeing_i,log10seeing_Z,log10seeing_Y,mag,EBV,R50,R50_gt,R50_rt,R50_Yt,R50_Jt,R50_Ht,R90,R100,N100,axrat,ang,groupID,Ngroup,mag_app_Zt,mag_Zt,mag_app_gt,mag_app_rt,mag_app_it,mag_app_Yt,flux_ut,flux_err_ut,flux_ut_uncorrected,flux_err_ut_uncorrected,flux_gt,flux_err_gt,flux_gt_uncorrected,flux_err_gt_uncorrected,flux_rt,flux_err_rt,flux_rt_uncorrected,flux_err_rt_uncorrected,flux_it,flux_err_it,flux_it_uncorrected,flux_err_it_uncorrected,flux_Zt,flux_err_Zt,flux_Zt_uncorrected,flux_err_Zt_uncorrected,flux_Yt,flux_err_Yt,flux_Yt_uncorrected,flux_err_Yt_uncorrected,flux_Jt,flux_err_Jt,flux_Jt_uncorrected,flux_err_Jt_uncorrected,flux_Ht,flux_err_Ht,flux_Ht_uncorrected,flux_err_Ht_uncorrected,flux_Kt,flux_err_Kt,flux_Kt_uncorrected,flux_err_Kt_uncorrected,flux_W1t,flux_err_W1t,flux_W1t_uncorrected,flux_err_W1t_uncorrected,flux_W2t,flux_err_W2t,flux_W2t_uncorrected,flux_err_W2t_uncorrected,flux_uc,flux_err_uc,flux_gc,flux_err_gc,flux_rc,flux_err_rc,flux_ic,flux_err_ic,flux_Zc,flux_err_Zc,flux_Yc,flux_err_Yc,flux_Jc,flux_err_Jc,flux_Hc,flux_err_Hc,flux_Kc,flux_err_Kc,flux_W1c,flux_err_W1c,flux_W2c,flux_err_W2c,mask,starmask,starscol,starssize,class,noOPT_r,noOPT_i,noIR_Z,noIR_Y,duplicate,mag_Zc,mag_Zt_fake,cluster_label,cluster_label_no_u,umap1_no_u,umap1_u,umap2_no_u,umap2_u,umap3_no_u,umap3_u,umap4_no_u,umap4_u,umap5_no_u,umap5_u,umap6_no_u,umap6_u,umap7_no_u,umap7_u,umap8_no_u,umap8_u,umap9_no_u,umap9_u,umap10_no_u,umap10_u,final_label,umap1,umap2,umap3,umap4,umap5,umap6,umap7,umap8,umap9,umap10,missing_bands
uberID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9580882500059,waves_postprocessed_10.0_-42.5.rds,958.0,96,8824.5,58.5,6675.881665,9.791838,-43.078226,9.791855,-43.078250,9.791777,9.791891,-43.078258,-43.078257,9.791801,-43.078256,9.791810,-43.078236,9.791741,-43.078250,9.791765,-43.078248,9.791741,-43.078250,9.791838,-43.078225,9.791855,-43.078250,-0.002034,0.002893,-0.123168,-0.119852,NaN,-0.120193,-0.166938,15.111749,0.013676,0.725778,0.852429,0.776101,0.675511,0.647937,0.653555,2.218231,11.149482,4120,0.949469,41.601561,96,11,15.797645,15.088464,18.469780,17.163015,16.416433,15.483403,2.882643e-05,3.618253e-07,2.713136e-05,3.405490e-07,3.513708e-04,2.522236e-06,3.355205e-04,2.408458e-06,1.155568e-03,4.160529e-07,1.117569e-03,4.023716e-07,0.002289,2.316756e-06,0.002230,2.256998e-06,0.003347,1.348588e-06,0.003282,1.322478e-06,0.004260,3.651365e-06,0.004195,3.596005e-06,0.004631,3.878294e-06,0.004580,3.835726e-06,0.005287,7.656555e-06,0.005249,7.602184e-06,0.004283,5.494495e-06,0.004263,5.469025e-06,0.001973,1.636403e-06,0.001968,1.632260e-06,0.001160,1.389919e-06,0.001158,1.387474e-06,2.679044e-05,2.647858e-07,3.409320e-04,1.008414e-06,1.148247e-03,3.282160e-07,2.284306e-03,1.430991e-06,3.333865e-03,5.246856e-07,4.241085e-03,1.441600e-06,4.614449e-03,1.555188e-06,5.255760e-03,3.291831e-06,4.260607e-03,2.424957e-06,1.856062e-03,4.583290e-07,1.064517e-03,4.881762e-07,0.0,0.0,3.0,3.0,star,0.0,0.0,0.0,0.0,1.0,15.092630,0,star,star,8.790334,8.958279,2.996169,2.727392,-0.242085,-0.264970,9.033234,0.730949,2.786047,6.635572,9.428096,2.042982,2.710304,1.106028,7.440554,8.583146,6.330934,3.740723,4.758306,5.383004,star,8.958279,2.727392,-0.264970,0.730949,6.635572,2.042982,1.106028,8.583146,3.740723,5.383004,False
9581275900715,waves_postprocessed_10.0_-42.5.rds,958.0,102,12758.5,714.5,8550.818440,9.343646,-43.021880,9.343656,-43.021902,9.343692,9.343692,-43.021909,-43.021909,9.343651,-43.021933,9.343593,-43.021931,9.343656,-43.021902,9.343615,-43.021925,9.343656,-43.021902,9.343647,-43.021879,9.343656,-43.021902,-0.000

In [39]:
df['uberID']=df.index

In [40]:
# Saving
df.to_csv('../data/WD01_UMAP_full.csv')

In [41]:
columns_to_save = ['uberID', 'final_label', 'missing_bands', 'umap1', 'umap2', 'umap3', 'umap4', 'umap5', 'umap6', 'umap7', 'umap8', 'umap9', 'umap10', 'class']

In [42]:
df[columns_to_save].to_csv('../data/WD01_UMAP_reduced_columns.csv')